In [2]:
import pandas as pd
import numpy as np

Fuente de Datos:

https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud

* Actividad: Descargar datos directamente con API de Kaggle o Consumir los datos desde un FASTAPI

## 1. Importar datos y validacion rapida de datos importados

In [ ]:
# De ser necesario, crear carpeta "data" en el Modulo3
# Incluir la ruta en maquina local o donde se carguen los datos

data_path = "C:/Users/Sebastian/Documents/Projects/Credibanco/Modulo3/data/creditcard.csv"
data = pd.read_csv(data_path)

In [6]:
# Display basic information
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [7]:
# Check for missing values
print(data.isnull().sum())

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64


## 2. Realizar EDA (Analisis exploratorio de datos) - Actividad

## 3. Realizar Transformacion de Variables y Feature Selection - Actividad

## 4. Entrenamiento y evaluacion del  modelo

In [ ]:
# Partición de datos en entrenamiento y visualizacion

from sklearn.model_selection import train_test_split

# Separate features and labels
X = data.drop('Class', axis=1)
y = data['Class']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Balanceo de clases

from imblearn.over_sampling import SMOTE

# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [11]:
import os
from sklearn.ensemble import RandomForestClassifier
import joblib

# Crear carpeta save_models si no existe
os.makedirs('save_models', exist_ok=True)

# Inicializar el modelo
model = RandomForestClassifier(n_estimators=20, random_state=42)

# Entrenar el modelo
model.fit(X_train_resampled, y_train_resampled)

# Guardar el modelo en la carpeta save_models
model_path = os.path.join('save_models', 'random_forest_model.joblib')
joblib.dump(model, model_path)

print(f"Modelo guardado en: {model_path}")


Modelo guardado en: save_models\random_forest_model.joblib


In [12]:
import joblib
from sklearn.metrics import classification_report, confusion_matrix

# Cargar el modelo guardado
model_path = os.path.join('save_models', 'random_forest_model.joblib')
model = joblib.load(model_path)

# Realizar predicciones
y_pred = model.predict(X_test)

# Mostrar métricas de evaluación
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[56846    18]
 [   18    80]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.82      0.82      0.82        98

    accuracy                           1.00     56962
   macro avg       0.91      0.91      0.91     56962
weighted avg       1.00      1.00      1.00     56962



Actividad: Experimentar con multiples modelos e hiperparametros para obtener un mejor modelo

Actividad 2: Establecer límites para que el agente los considere

## 5. Langchain

In [13]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)

openai_model = "gpt-4o-mini"

from langchain_openai import ChatOpenAI

# For normal accurate responses
llm = ChatOpenAI(temperature=0.0, model=openai_model)

In [14]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["transaction_details"],
    template="Given the following transaction details, explain why it might be considered fraudulent:\n{transaction_details}"
)

# Create the LLMChain
fraud_explainer = LLMChain(llm=llm, prompt=prompt_template)

C:\Users\Sebastian\AppData\Local\Temp\ipykernel_23632\448211833.py:11: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  fraud_explainer = LLMChain(llm=llm, prompt=prompt_template)


In [15]:
# Example transaction details
transaction_details = """
Transaction ID: 123456
Amount: $5000
Time: 02:30 AM
Location: Unknown
Previous Transactions: None
"""

# Generate explanation
explanation = fraud_explainer.run(transaction_details)
print(explanation)

C:\Users\Sebastian\AppData\Local\Temp\ipykernel_23632\1504283823.py:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  explanation = fraud_explainer.run(transaction_details)


The transaction details provided raise several red flags that could indicate potential fraud:

1. **High Transaction Amount**: A transaction of $5000 is relatively large, especially if there are no previous transactions associated with the account. This could suggest that the account is being used for a significant purchase or withdrawal that is inconsistent with the account holder's typical behavior.

2. **Time of Transaction**: The transaction occurred at 2:30 AM, which is an unusual time for financial transactions. Most legitimate transactions occur during regular business hours or when the account holder is likely to be awake and active. Late-night transactions can be a sign of unauthorized access or fraudulent activity.

3. **Location**: The transaction is marked as occurring in an "unknown" location. This lack of specific location information can be suspicious, as legitimate transactions typically have identifiable locations. If the transaction was made from a location that is no

Actividad: Añadir funcionalidad vista en el modulo de conceptos de Langchain